# Informe Modelo Predictivo Popularidad de Canciones en Spotify
<p style="text-align: justify;">
Este informe presenta el desarrollo de un modelo de aprendizaje automático, cuyo ibjetivo es predecir el nivel de popularidad de canciones en Spotify. A lo largo del documento se describen las etapas fundamentales del proceso, incluyendo el preprocesamiento de datos, la selección y calibración del modelo, el entrenamiento y evaluación del rendimiento, y la disponibilización del modelo mediante una API. El enfoque se basa en el uso de modelos de árboles de decisión y ensamblajes, implementados con Python.<p>

In [ ]:
## Librerias a Importar
import warnings
warnings.filterwarnings('ignore')
# Manipulación de datos
import pandas as pd
import numpy as np
# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Modelos base
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    StackingRegressor
)
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
# Preprocesamiento y selección de características
from sklearn.feature_selection import SelectFromModel

## Preprocesamiento de Datos
<p style="text-align: justify;">
En este proyecto se usaró el conjunto de datos de datos de popularidad en canciones, donde cada observación representa una canción y se tienen variables como: duración de la canción, acusticidad y tempo, entre otras. El objetivo es predecir qué tan popular es la canción.<p>

In [2]:
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/dataTrain_Spotify.csv')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/dataTest_Spotify.csv', index_col=0)

In [3]:
print(tabulate(dataTraining.head(), headers='keys', tablefmt='psql'))

+----+--------------+------------------------+-------------------+----------------------+----------------------------------------+---------------+------------+----------------+----------+-------+------------+--------+---------------+----------------+--------------------+------------+-----------+---------+------------------+---------------+--------------+
|    |   Unnamed: 0 | track_id               | artists           | album_name           | track_name                             |   duration_ms | explicit   |   danceability |   energy |   key |   loudness |   mode |   speechiness |   acousticness |   instrumentalness |   liveness |   valence |   tempo |   time_signature | track_genre   |   popularity |
|----+--------------+------------------------+-------------------+----------------------+----------------------------------------+---------------+------------+----------------+----------+-------+------------+--------+---------------+----------------+--------------------+------------+--

<p style="text-align: justify;">
A continuación, se valida la dimensión de los datos, los tipos de variables y se indentifica la existencia de valores faltantes y duplicados. Según los resultados, se identificaron valores duplicados y algunas variables string y categoricas. De igual forma se identifica que existen variables tipo <code>int64</code> y <code>float64</code>, los cuales podrían generan un uso importante de la memoria.<p>

In [4]:
# Revisión inicial de datos
print("Dimensión de los datos de entrenamiento:", dataTraining.shape)

if dataTraining.isnull().sum().sum() > 0: # Verifica si hay valores nulos
    print("\nValores nulos en los datos de entrenamiento:")
    print(dataTraining.isnull().sum())
else:
    print("\nNo hay valores nulos en los datos de entrenamiento.")

print("\nNúmero de canciones duplicadas según Track_ID:")
print(dataTraining['track_id'].duplicated().sum())

print("\nTipos de variables en la base de datos:")
print(dataTraining.dtypes.unique())

print("\nVariables string y categóricas en la base de datos:")
print(dataTraining.select_dtypes(include=['object', 'bool']).info())

Dimensión de los datos de entrenamiento: (79800, 21)

No hay valores nulos en los datos de entrenamiento.

Número de canciones duplicadas según Track_ID:
13080

Tipos de variables en la base de datos:
[dtype('int64') dtype('O') dtype('bool') dtype('float64')]

Variables string y categóricas en la base de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79800 entries, 0 to 79799
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   track_id     79800 non-null  object
 1   artists      79800 non-null  object
 2   album_name   79800 non-null  object
 3   track_name   79800 non-null  object
 4   explicit     79800 non-null  bool  
 5   track_genre  79800 non-null  object
dtypes: bool(1), object(5)
memory usage: 3.1+ MB
None


<p style="text-align: justify;">
A continuación, se realizarán algunos ajustes a la base de datos. En primer lugar se optimiza el uso de memoria del dataset <code>dataTraining</code> ajustando los tipos de datos según su contenido. Este procedimiento es relevante porque reduce significativamente el consumo de memoria, mejorando la eficiencia del procesamiento y el rendimiento del modelo. Posteriormente, se eliminan las columnas que no se usarán y crearán nuevas variables que serán incluidas en el modelo.<p>

In [5]:
# Eliminación de duplicados
dataTraining = dataTraining.drop_duplicates(subset='track_id', keep='first')

In [6]:
# Transformación tipológica de variables con el objetivo de reducir el uso de memoria
# Se recomienda usar float32 en lugar de float64 y int32 en lugar de int64
for col in dataTraining.columns:
    col_type = dataTraining[col].dtype

    if col_type == 'float64':
        dataTraining[col] = dataTraining[col].astype('float32')
    elif col_type == 'int64':
        dataTraining[col] = dataTraining[col].astype('int32')
    elif col_type == 'bool':
        dataTraining[col] = dataTraining[col].astype('int8')

In [7]:
# Creación de variables nuevas
# Se crean variables que pueden ser útiles para el modelo
for df in [dataTraining, dataTesting]:
    df['track_name_length'] = df['track_name'].apply(lambda x: len(str(x)))
    df['tempo_density'] = df['tempo'] / df['duration_ms']
    df['energy_danceability'] = df['energy'] * df['danceability']
    df['acousticness_bin'] = (df['acousticness'] > 0.5).astype(int)

In [8]:
# Eliminación de variables string
for col in ['Unnamed: 0']:
    if col in dataTraining.columns: dataTraining.drop(columns=col, inplace=True)
    if col in dataTesting.columns: dataTesting.drop(columns=col, inplace=True)

drop_cols = ['track_id','artists','album_name','track_name', 'track_genre']
X = dataTraining.drop(columns=drop_cols + ['popularity'])
y = dataTraining['popularity']
XTesting = dataTesting.drop(columns=drop_cols)

<p style="text-align: justify;">
Adicionalmente, como parte del pre procesamiento de datos se realiza una selección de variables utilizando un modelo de Ranfom Forest como base. Este modelo perimite seleccionar las características según su nivel de relevancia. De acuerdo con los resultados obtenidos se seleccionaron 13 variables de las cuales 10 vienen de la base de datos original y 3 fueron creadas en pasos previos.<p>

In [9]:
# Selección de variables usando Random Forest
selector = SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42))
selector.fit(X, y)
X_sel = selector.transform(X)
X_test_sel = selector.transform(XTesting)

In [13]:
# Variables predictoras
selected_columns = X.columns[selector.get_support()].tolist()
print(tabulate([[col] for col in selected_columns], headers=["Columnas Seleccionadas"], tablefmt="psql"))

+--------------------------+
| Columnas Seleccionadas   |
|--------------------------|
| duration_ms              |
| danceability             |
| energy                   |
| loudness                 |
| speechiness              |
| acousticness             |
| instrumentalness         |
| liveness                 |
| valence                  |
| tempo                    |
| track_name_length        |
| tempo_density            |
| energy_danceability      |
+--------------------------+


## Análisis de Estadísticas Descriptivas
<p style="text-align: justify;">
En esta sección se realiza un análisis descriptivo de la base de datos. Se presentan las principales estadísticas descriptivas, así como histogramas, correlaciones y visualización de valores atípicos.<p>


In [ ]:
# Visualización datos de entrenamiento
dataTraining.describe()

In [ ]:
# Estilos de los gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Histograma de variables numéricas
dataTraining.select_dtypes(include=np.number).hist(bins=30, figsize=(18, 12), color='skyblue', edgecolor='black')
plt.suptitle("Distribución de variables numéricas", fontsize=18)
plt.tight_layout()
plt.show()

# Mapa de calor de correlación
plt.figure(figsize=(14, 10))
corr = dataTraining.select_dtypes(include=np.number).corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title("Matriz de Correlación", fontsize=16)
plt.show()

# --- 3. Boxplots para identificar outliers ---
for col in dataTraining.select_dtypes(include=np.number).columns:
    plt.figure(figsize=(10, 4))
    sns.boxplot(x=dataTraining[col], color='lightgreen')
    plt.title(f"Boxplot: {col}")
    plt.show()


In [ ]:
# Variables categóricas
cat_cols = dataTraining.select_dtypes(include=['object','category']).columns.tolist()

# Visualizar la cantidad de valores únicos por variable categórica
print("Valores únicos por variable categórica:")
for col in cat_cols:
    print(f"{col}: {dataTraining[col].nunique()} valores únicos")

# Distribución de frecuencia de las principales categorías
for col in cat_cols:
    if dataTraining[col].nunique() <= 50:  # Limita el análisis a columnas con un número manejable de categorías
        plt.figure(figsize=(10, 5))
        dataTraining[col].value_counts().head(20).plot(kind='bar', color='cornflowerblue')
        plt.title(f"Top 20 valores de {col}")
        plt.ylabel("Frecuencia")
        plt.xlabel(col)
        plt.xticks(rotation=45)
        plt.show()

# Popularidad promedio por categoría
for col in cat_cols:
    if 'popularity' in dataTraining.columns and dataTraining[col].nunique() <= 50:
        popularity_by_cat = dataTraining.groupby(col)['popularity'].mean().sort_values(ascending=False).head(20)
        plt.figure(figsize=(10, 5))
        sns.barplot(x=popularity_by_cat.values, y=popularity_by_cat.index, palette="viridis")
        plt.title(f"Popularidad promedio por {col} (Top 20)")
        plt.xlabel("Popularidad Promedio")
        plt.ylabel(col)
        plt.show()


In [ ]:
# Agrupar por 'track_genre' y calcular las medias de las columnas seleccionadas
perfil_generos = dataTraining.groupby('track_genre')[dataTraining.select_dtypes(include=np.number).columns].mean().round(2).sort_values('popularity', ascending=False)

# Mostrar el perfil
print(perfil_generos)

## Separación de Datos en Entrenamiento y Prueba
<p style="text-align: justify;">
Se dividió el conjunto de datos en dos subconjuntos: uno de entrenamiento y otro de validación, utilizando una proporción del 80 % para entrenamiento y 20 % para validación.<p>

In [16]:
# Dividir los datos en entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X_sel, y, test_size=0.2, random_state=42)

## Selección y Calibración del Modelo

#### (a) Selección del Modelo Predictivo
<p style="text-align: justify;">
A partir de un proceso iterativo de prueba y error con diferentes modelos predictivos, se eligió el modelo de Satcking como el modelo definitivo para participar en la competencia de predicción del nivel de popularidad de las cacniones en Spotify. A continuación se presentarán el procedimiento de calibración, entrenamiento, predicción y evaluación del desempeño del mismo.<p>

#### (b) Calibración de Modelos Individuales

In [ ]:
# Calibrar Random Forest
rf_params = {'n_estimators': [100, 200], 'max_depth': [20, 30]}
rf_grid = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=3)
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_

In [ ]:
# Calibrar Gradient Boosting
gb_params = {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.075], 'max_depth': [3, 5]}
gb_grid = GridSearchCV(GradientBoostingRegressor(random_state=42), gb_params, cv=3)
gb_grid.fit(X_train, y_train)
best_gb = gb_grid.best_estimator_

In [ ]:
# Calibrar XGBoost
xgb_params = {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.075], 'max_depth': [3, 5]}
xgb_grid = GridSearchCV(XGBRegressor(random_state=42), xgb_params, cv=3)
xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_

#### (c) Entrenamiento del Modelo Stacking

In [ ]:
# Definir los modelos base
base_models = [
    ('rf', RandomForestRegressor(n_estimators=200, max_depth=30, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=200, learning_rate=0.075, max_depth=5, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=200, learning_rate=0.075, max_depth=5, random_state=42))
]

In [ ]:
# Definir el modelo de stacking
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, max_depth=3),
    passthrough=True,
    n_jobs=-1,
    cv=5
)

In [ ]:
# Entrenar el modelo de stacking con los datos transformados
stacking_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_pred = stacking_model.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE validación local: {rmse:.5f}")

In [ ]:
# Entrenar el modelo de stacking con los datos transformados
stacking_model.fit(X_train, y_train)

# Generar predicciones finales en el conjunto de prueba
test_pred = stacking_model.predict(X_test_sel)
submission = pd.DataFrame({'ID': dataTesting.index, 'popularity': test_pred})

In [ ]:
# Crear el archivo de envío para Kaggle
submission.to_csv('test_submission_file.csv', index=False)
submission.head()

#### (d) Evaluación del Desempeño

In [ ]:
# Evaluación del modelo
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {RMSE:.5f}")

## Disponibilización del Modelo

## Conclusiones